In [2]:
!pip install GEOparse pandas scipy statsmodels

In [37]:
import os
import gzip
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

# Set correct path to your extracted folder
extract_path = "/Users/nehaeshwaragari/Documents/miBiome Internship/GSE64850_RAW"

# Define files
metastasis_files = [
    "GSM1581605_67161.fpkm.txt.gz",
    "GSM1581606_84816.fpkm.txt.gz",
    "GSM1581607_94948.fpkm.txt.gz"
]
no_metastasis_files = [
    "GSM1581608_76948.fpkm.txt.gz",
    "GSM1581609_86923.fpkm.txt.gz",
    "GSM1581610_94812.fpkm.txt.gz"
]
file_paths = [os.path.join(extract_path, f) for f in metastasis_files + no_metastasis_files]

# Read and merge expression data
expression_data = []
for file_path in file_paths:
    with gzip.open(file_path, 'rt') as f:
        df = pd.read_csv(f, sep="\t")
        df = df[['gene_short_name', 'FPKM']].copy()
        df.columns = ['Gene', os.path.basename(file_path).split("_")[0]]
        expression_data.append(df)

# Merge on 'Gene'
merged_df = expression_data[0]
for df in expression_data[1:]:
    merged_df = pd.merge(merged_df, df, on='Gene')

merged_df = merged_df[~merged_df['Gene'].duplicated(keep='first')]
merged_df.set_index('Gene', inplace=True)
merged_df = merged_df.apply(pd.to_numeric, errors='coerce')

# Groups
group1 = ['GSM1581605', 'GSM1581606', 'GSM1581607']
group0 = ['GSM1581608', 'GSM1581609', 'GSM1581610']

# T-test and fold change
p_values = []
log2fc = []
genes = []

for gene in merged_df.index:
    vals1 = merged_df.loc[gene, group1].values
    vals0 = merged_df.loc[gene, group0].values
    if np.isnan(vals1).any() or np.isnan(vals0).any():
        continue
    stat, p = ttest_ind(vals1, vals0, equal_var=False)
    p_values.append(p)
    log2fc.append(np.log2(np.mean(vals1) + 1e-6) - np.log2(np.mean(vals0) + 1e-6))
    genes.append(gene)

# Create results dataframe
results = pd.DataFrame({
    'Gene': genes,
    'log2FC': log2fc,
    'p-value': p_values
})

# Exploratory filter: raw p-value + effect size
exploratory = results[(results['p-value'] < 0.05) & (np.abs(results['log2FC']) > 0.5)]
exploratory = exploratory.sort_values('p-value')

# Save and show
exploratory.to_csv("exploratory_gene_list.csv", index=False)
print("Top exploratory genes (p < 0.05 and |log2FC| > 0.5):")
print(exploratory.head(10))

Top exploratory genes (p < 0.05 and |log2FC| > 0.5):
           Gene    log2FC   p-value
2452       RERG -2.437042  0.000070
11319  SLC25A25  0.865500  0.000178
8198      RNF13 -1.002352  0.000404
8818     SORBS2 -3.522439  0.000413
10330     RABL5 -1.899339  0.000417
4761     DUSP14 -1.704473  0.000427
283      ZBTB8A -1.528991  0.000493
8721      MAML3 -1.678754  0.000513
9241      NDST1 -1.300696  0.000519
4195       XPO6  3.445299  0.000550


#### Here’s the code that generates a meaningful exploratory gene list by:

- Using raw p-value < 0.05

- Relaxing log2FC threshold to 0.5

- Returning top genes even if not statistically significant after correction

